# Домашнее задание 1. Разрешение семантической многозначности (WSD).

При выполнении домашнего задания можно пользоваться тетрадками с семинаров. Пожалуйста, соблюдайте минимальную гигиену кода:
* пишите комментарии, где они уместны;
* избегайте вывода больших текстовых переменных (например, не стоит печатать весь корпус, это существенно усложняет читаемость вашей тетрадки);
* запускайте ячейки последовательно, иначе можно получить невоспроизводимые результаты.

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Описание задания:

В этом задании вам нужно будет кластеризовать контексты для неоднозначных слов и проанализировать, как результаты соотносятся с их значениями. Напоминаем, что у нас могут быть как многозначные слова, так и омонимы - для сравнения хорошо взять и те, и другие.

Для сбора данных в п. 1 предполагается использование краулинга. Советуем начать с ручного сбора данных, а затем, если останется время, сделать краулинг словаря. Так вы рискуете меньшим количеством баллов ;)

1. Выберите 5 неоднозначных лексем и соберите для них словарные значения (толкования). Можно использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/) или любой другой толковый словарь. Обязательно укажите, каким словарем вы пользовались. - 1 балл за ручной сбор данных, 1 балл за краулинг

Я взял грамоту.ру, потому что не хочу поднимать селениум. Я думаю, сомнений в том, что я умею нажимать ctrl+c, ctrl+v, нет (но ладно, я скопировал), поэтому сразу краулер. Слова, которые я взял - внизу, ссылка тоже

In [ ]:
РУЧКА
РУЧК|А, -и, род., мн. ручек, ж., нд., III в.
● 1.0. уменьш.-ласк. к рука 1.0., маленькая рука. Худенькая р. Мягкая р. Тёплая р. Изящная р. Ручки младенца.

2.0. Часть предмета, за к-рую его держат или берут рукой при совершении разного рода действий.   Син. скобка. Ср. рукоятка.

[Не]большая (маленькая, широкая, узкая, тонкая, толстая, круглая, длинная, короткая, [не]прочная, крепкая, надёжная, хлипкая разг., [не]удобная, кожаная, металлическая, медная, пластмассовая, деревянная, блестящая, поли

In [1]:
import urllib.request
import requests
import re
import os
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

import pandas as pd
import numpy as np

session = requests.session()
session.trust_env = False

In [2]:
def clean_entry(entry):
    # там сложно что-то нормально почистить, я забил
    entry = str(entry)
    if "<span" in entry:
        entry = entry.split(". <span")[0]
    # if "</i>" in entry:
    #     entry = entry.split("</i>. ")[1]
    # else:
    #     entry = entry.split(". ")[1]
    return re.sub("<.*?>", "", entry)

words = np.array(["идти", "земля", "ручка", "общество", "ум"])[:, np.newaxis]

In [3]:
word_dict = []

for word in tqdm(words):

    # идём по ссылке
    word = word[0]
    url = f"https://gramota.ru/poisk?query={word}&mode=slovari&dicts[]=48"
    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    # текст собираю по всем <p>
    desc = soup.find("div", {"class": "description highlightable"})
    all_entries = desc.find_all("p")
    all_entries = [
        clean_entry(entry)
        for entry in all_entries
        if re.match(r"(.{2}?)\b\d\.", entry.text)
    ]
    for idx, entry in enumerate(all_entries):
        word_dict.append({"word": word, "context": entry, "meaning": idx})
        
word_dict = pd.DataFrame(word_dict)

  0%|          | 0/5 [00:00<?, ?it/s]

2. Выберите один или несколько корпусов [коллекции Corus](https://github.com/natasha/corus#reference), извлеките оттуда все предложения, где содержатся ваши неоднозначные слова. - 1 балл

In [4]:
# %%capture
# !pip install -q corus pymorphy3 tensorflow Cython matplotlib simple-elmo
# !curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib
# !pip install -q git+https://github.com/lopuhin/python-adagram.git
# !wget http://vectors.nlpl.eu/repository/20/196.zip
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [5]:
from corus import load_lenta
from pymorphy3 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("\w+")


def lemmatize(text):
    res = [
        morph.parse(word)[0].normal_form
        for word in tokenizer.tokenize(text)
    ]
    return res


path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
morph = MorphAnalyzer()
contexts = [next(records).text for _ in range(50000)]

In [6]:
from joblib import Parallel, delayed

def extract_context(record):

    # сразу леммы, потому что это имеет больше смысла
    lemmatized = np.array(lemmatize(record))
    in_context = np.in1d(words, lemmatized)
    res = np.hstack([
         words, np.vstack([" ".join(lemmatized)]*5)
    ])
    return res[in_context]


# в колабе всего 2 ядра, серьёзно? ладно...
word_context = np.vstack(
    Parallel(48, backend="multiprocessing")(
    delayed(extract_context)(record)
    for record in tqdm(contexts)
))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [7]:
meanings = pd.DataFrame(word_context, columns=["word", "context"])
meanings.head()

,word,context
0,земля,полицейский алекс коллинс и трейся холлоуэй ок...
1,идти,на митинг верующий проходить в киев в время об...
2,идти,президент украина пётр порошенко занять место ...
3,земля,в русский православный церковь рпц назвать кан...
4,идти,абсолютный чемпион мир в первый тяжёлый вес ук...


In [8]:
word_dict["context"] = word_dict.context.apply(
    lambda y: " ".join(x for x in lemmatize(y))
)
meanings = pd.concat([meanings, word_dict])
meanings

,word,context,meaning
0,земля,полицейский алекс коллинс и трейся холлоуэй ок...,NaN
1,идти,на митинг верующий проходить в киев в время об...,NaN
2,идти,президент украина пётр порошенко занять место ...,NaN
3,земля,в русский православный церковь рпц назвать кан...,NaN
4,идти,абсолютный чемпион мир в первый тяжёлый вес ук...,NaN
...,...,...,...
22,общество,4 0 группа человек к рые вместе проводить время,3.0
23,общество,5 0 объединение человек как правило добровольн...,4.0
24,ум,1 0 зд ед способность человек думать лежать в ...,0.0
25,ум,1 1 зд ед высокий степень такой способность вы...,1.0


3. Разбейте полученные контексты по значениям двумя любыми способами (вы можете использовать способы с семинара, заменить там эмбеддинги, реализовать какой-то свой способ).
Напоминалка: в случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo и другими контекстуальными эмбеддингами — нужно кластеризовать. - 2 балла

Это уже было на компсеме, кстати. Я попробую через эмбеддинги и adagram. Там всё равно ничего прикольного не получится, так что метод не имеет значения. Попробую через гауссовские смеси - гипотеза в том, что несколько значений будут иметь какие-то собственные распределения, будем пытаться их вычленить из общей каши

In [ ]:
import adagram
from simple_elmo import ElmoModel

vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')
elmo = ElmoModel()
elmo.load("196.zip", max_batch_size=256)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.mixture import GaussianMixture

meanings["cluster_meaning"] = np.nan
# вот это самый важный момент на самом деле, но
# мне лень экспериментировать с тсне и его параметрами
svd = TruncatedSVD(2, random_state=69)

for word in word_dict.word.unique():
    
    word_mask = meanings.word == word
    # там уже есть опция доставать средний эмбеддинг контекста, зачем мудрить
    context_embs = elmo.get_elmo_vector_average(
        meanings[word_mask]["context"].apply(lambda x: x.split()),
        layers="top", warmup=False
    )
    
    mapped_meanings = svd.fit_transform(context_embs)
    n_of_meanings = word_dict[word_dict.word == word].meaning.nunique()
    gauss = GaussianMixture(n_components=n_of_meanings, random_state=69)
    clusters = gauss.fit_predict(mapped_meanings)
    meanings.loc[word_mask, "cluster_meaning"] = clusters

In [22]:
%%capture

meanings["adagram_meaning"] = np.nan
for idx, row in meanings.iterrows():
    word, context = row[["word", "context"]]
    meanings.loc[idx, 'adagram_meaning'] = vm.disambiguate(word, context).argmax()

In [23]:
meanings.head()

,word,context,meaning,cluster_meaning,adagram_meaning
0,земля,полицейский алекс коллинс и трейся холлоуэй ок...,NaN,5.0,4.0
1,идти,на митинг верующий проходить в киев в время об...,NaN,3.0,0.0
2,идти,президент украина пётр порошенко занять место ...,NaN,6.0,4.0
3,земля,в русский православный церковь рпц назвать кан...,NaN,5.0,4.0
4,идти,абсолютный чемпион мир в первый тяжёлый вес ук...,NaN,3.0,4.0


In [24]:
meanings.to_csv("meanings.csv", index=0)

4. Возьмите словарные толкования для каждого значения и припишите им значение автоматически
теми же способами, что и в п.2. - 1 балл

Что и в пункте 3, я надеюсь, потому что в пункте 2 я ничего такого не делал. Но это надо делать там же, потому что модельки я не сохраняю, так что см. выше

5. Выберите по 2-3 значения для слова и по 5 контекстов, которые автоматически были отнесены к этим значениям. Разметьте контексты: совпадает ли значение слова в контексте со словарным значением. - 1 балл

In [115]:
chosen_contexts = pd.DataFrame(columns=[
    "word", "context", "adagram_meaning", "cluster_meaning"
])

idx = 0
for word in word_dict.word.unique():
    most_popular_meanings = meanings[
        meanings.word == word
    ].adagram_meaning.value_counts().index[:2]
    for meaning in most_popular_meanings:
        contexts = meanings[
            (meanings.word == word) &
            (meanings.adagram_meaning == meaning)
        ]
        # у меня не везде есть 5, некоторые значения очень редкие
        num_samples = min(contexts.shape[0], 5)
        contexts = contexts.sample(num_samples)
        for _, context in contexts.iterrows():
            context_raw = context["context"]
            cluster_meaning = context["cluster_meaning"]
            chosen_contexts.loc[idx] = (word, context_raw, meaning, cluster_meaning)
            idx += 1

In [146]:
chosen_contexts.to_csv("chosen_contexts.csv", index=0)

Не понимаю. Кластеризация мне не говорит, какие именно значения она выделяет. Я просто посмотрю, что значения в тех контекстах, что выделились, однородны

Я разметил ручками все выбранные контексты. Если у меня есть значение среди толкований, я приписывал соответствующий индекс, если нет, например, это фразеологизм, то ставил -1. Можно проверить, что в словосочетаниях с похожим смыслом метка значения тоже похожа

In [154]:
my_meanings = """-1
-1
-1
4
-1
-1
3
4
1
4
1
4
4
3
3
1
2
2
2
2
2
1
2
2
2
2
2
3
2
3
3
-1
-1
-1
-1
-1
-1
-1
2
3
1"""

In [153]:
chosen_contexts["meaning"] = my_meanings.split("\n")

In [158]:
for idx, row in chosen_contexts.iterrows():
    
    word, context, meaning = row[["word", "context", "meaning"]]
    context = np.array(context.split())
    index = np.where(context == word)[0][0]
    window = np.arange(max(0, index-5), min(len(context), index+6))
    print(word, context[window], meaning)

идти ['цензура' 'здесь' 'абсолютно' 'речь' 'не' 'идти' 'потому' 'что' 'весь'
 'информация' 'она'] -1
идти ['неприличный' 'и' 'грубый' 'ведь' 'речь' 'идти' 'о' 'конкретный'
 'заявление' 'обама' 'это'] -1
идти ['в' 'русало' 'о' 'какой' 'публикация' 'идти' 'речь' 'не' 'поясняться'
 'распространять' 'агентство'] -1
идти ['год' 'между' 'сша' 'и' 'китай' 'идти' 'торговый' 'война' 'который'
 'выражаться' 'в'] 4
идти ['профинансировать' 'значительный' 'часть' 'транзакция' 'речь' 'идти' 'о'
 'продажа' '19' '5' 'процент'] -1
идти ['по' 'данные' 'риа' 'новость' 'речь' 'идти' 'о' 'красный' 'икра' 'в'
 'мвд'] -1
идти ['60' 'процент' 'весь' 'запрос' 'далее' 'идти' 'надувной' 'лодка' 'затем'
 'весельный' 'катер'] 3
идти ['в' 'настоящий' 'время' 'на' 'судно' 'идти' 'работа' 'по'
 'дооборудование' 'на' 'борт'] 4
идти ['жириновский' '6' '94' 'процент' 'далёкий' 'идти' 'ксения' 'собчак'
 'гражданский' 'инициатива' '2'] 1
идти ['сообщить' 'что' 'последний' 'несколько' 'год' 'идти' 'развитие'
 'онкологичес

6. Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает
с автоматически определенным значением толкования. - 1 балл

Вот теперь совсем ничего не понимаю. Толкование это одно, это тоже некоторый контекст, так сказать. Обычный контекст это другое. Я могу конечно сравнить, как были размечены толкования, проверить, совпадает ли их метка кластера с меткой кластера контекста, в котором это значение есть, но всё равно очень странно. Что мне должно дать аккураси? Даже если бы я сравнивал метки, которые я ставил вручную и метки, которые мне выдали кластеры, аккураси там ничего не даёт, у нас вообще-то разные метки, я ещё должен буду вычислить, а что же там за метку мне выдала кластеризация. Нет, нам интересно их распределение, тут либо рок-аук, либо BCubed, либо NMI. В статье про адаграм, которую кидали на семинаре, тоже считается именно NMI, он же V-measure, поэтому не вижу никаких причин считать аккураси. Там ещё и дисбаланс будет, потому что значений может быть много, это вообще харам

Посчитаем NMI, потому что нет гарантии, что 1 в тру и 1 в пред одинаковые, ранжирование тут уместнее. Ну либо рок-аук, но вроде в склёрне нет прикольного обобщения на многомерный случай, когда объём под поверхностью

In [159]:
from sklearn.metrics import normalized_mutual_info_score

print(normalized_mutual_info_score(
    chosen_contexts["meaning"],
    chosen_contexts["adagram_meaning"].astype(float),
))

0.25598769295361296


In [161]:
from sklearn.metrics import accuracy_score

print(accuracy_score(
    chosen_contexts["meaning"],
    chosen_contexts["adagram_meaning"].astype(float),
))

0.0


7. Сравните результаты двух методов WSD, объясните, почему результаты совпадают или расходятся (и покажите на конкретных примерах) - максимум 2 балла


Да я могу точно такую же метрику посчитать

In [162]:
print(normalized_mutual_info_score(
    chosen_contexts["meaning"],
    chosen_contexts["cluster_meaning"].astype(float),
))

0.31583854191150207


Если судить по метрикам, то мой метод с кластеризацией даже круче. Но изначально мы не можем знать и выводов мы тоже сделать не можем, почему один метод сработал лучше, а другой хуже. 

1. Я не имею ни малейшего понятия о том, как работает адаграм, я не знаю, почему он должен быть плох или хорош. Поскольку в статье написано про ЕМ-алгоритм, я даже прикасаться к этой дряни не хочу, мне уже хватило. Собственно и w2v я не уважаю и не понимаю, с чего бы вдруг он всегда говорил, какие слова похожи. Ну и поскольку там ЕМ-алгоритм, это статистический метод, а не нейросетка, нет гарантий, что мы действительно что-то учимся предсказывать, а не тупо запоминаем априорные распределения. Все баесовские подходы от этого страдают - если данных мало, то мы распределение выучить не сможем, могу предположить, что данных было слишком мало, у меня почти везде максимум 4 значения, а даже у слова "идти" их 10+, поэтому плохо


2. Почему кластеризация могла не сработать ответить легко - она не обязана размечать мне те смыслы, которые я от неё хочу. Может так получиться, что одно значение разбилось на 2 в результате работы алгоритма, может быть выделились какие-то другие значения, а не те, что я хотел, может быть неудачно выбран алгоритм, может быть неправильная проекция, может быть неправильные эмбеддинги, причин масса. Как исправить - без понятия, собственно это задача то не самая простая. Ну и качество не сказать, что совсем ужасное, что-то всё-таки удаётся понять, я бы вообще ожидал метрику в районе 0.1 или 0.001, это типично для таких задач. Можно ещё взять побольше данных и вообще нормально их предобработать - у меня там всякий мусор, сокращения, контексты слишком большие и т.д. Ещё я мог накосячить в ручной разметке, потому что не дружу с семантикой

А почему там результаты разошлись это вообще мрак, мне тогда нужно смотреть на $x$ в $P(y|x,\theta)$, но я же не могу ни один метод проинтерпретировать - все эмбеддинги кодируют свою информацию, разве что через shap какой-нибудь, но зачем? Ниже конкретные примеры, где что пошло так / не так

Сделаю только для слова "земля", потому что там есть все значения - нет -1, их в целом немного и иначе я просто сам запутаюсь, где какое значение, я напоминаю, у меня не совпадают метки. Хотя это можно исправить тем же ЕМом, но я не хочу, пожалуйста

In [167]:
for idx, row in chosen_contexts.iterrows():
    
    word, context, meaning = row[["word", "context", "meaning"]]
    if word == "земля":
        context = np.array(context.split())
        index = np.where(context == word)[0][0]
        window = np.arange(max(0, index-5), min(len(context), index+6))
        print(word, context[window])
        display(pd.DataFrame(row[["meaning", "adagram_meaning", "cluster_meaning"]]).T)

земля ['7' 'миллиард' 'световой' 'год' 'от' 'земля' 'в' 'созвездие' 'золотой'
 'рыба' 'adfs']


,meaning,adagram_meaning,cluster_meaning
10,1,4.0,0.0


земля ['с' 'воздух' 'и' 'возможно' 'на' 'земля' 'несколько' 'месяц' 'назад' 'мы'
 'уже']


,meaning,adagram_meaning,cluster_meaning
11,4,4.0,1.0


земля ['пострадать' 'мужчина' 'поднять' 'лежать' 'на' 'земля' 'предмет' 'он'
 'взорваться' 'и' 'ранить']


...

,meaning,adagram_meaning,cluster_meaning
13,3,4.0,3.0


земля ['milk' 'порядок' '11' 'тысяча' 'гектар' 'земля' 'под' 'строительство'
 'молочный' 'комплекс' 'th']


,meaning,adagram_meaning,cluster_meaning
14,3,4.0,1.0


земля ['открытый' 'космос' 'быть' 'отправить' 'на' 'земля' 'где' 'они' 'в'
 'течение' 'неделя']


,meaning,adagram_meaning,cluster_meaning
15,1,0.0,4.0


Что мы видим:

1. Значение "планета". У обоих алгоритмов разные ответы, они не вычленили это значение, кластер разнородный. Плохо
3. Значение "то, по чему ходим". Адаграм дал одно значение, но кластер неоднородный. Разнородный кластер у кластеризации. Плохо
4. Значение "территориальный участок". Одно значение по адаграму, опять неоднородный кластер. Кластеризация не вычленила. Плохо

По крайней мере кластеры от гаусса более разнообразные, это тоже важно для метрики, даже не пересекаются почти

Опять же, причин может быть масса почему так получается. Разгадка одна - unsupervised методы это трата времени. Пытаться что-то хорошее получить без таргета нереально. Если что, я могу рандомные векторы взять вместо эмбеддингов берта и всё равно получить "вменяемое" значение, проверим?

In [173]:
key_to_index = {}

idx = 0
for context in meanings["context"]:
    for word in context.split():
        if word not in key_to_index:
            key_to_index[word] = idx
            idx += 1    
            
emb_W = np.random.uniform(-0.5, 0.5, size=(idx, 100))

In [184]:
def aggregate_embs(string):
    return np.mean([emb_W[key_to_index[t]] for t in string.split()], axis=0)

In [196]:
svd = TruncatedSVD(2, random_state=69)

for word in word_dict.word.unique():
    
    word_mask = meanings.word == word
    context_embs = np.vstack(meanings[word_mask]["context"].apply(aggregate_embs).values)
    mapped_meanings = svd.fit_transform(context_embs)
    
    n_of_meanings = word_dict[word_dict.word == word].meaning.nunique()
    gauss = GaussianMixture(n_components=n_of_meanings, random_state=69)
    gauss.fit(mapped_meanings)
    
    # copy, paste...
    word_mask = chosen_contexts.word == word
    context_embs = np.vstack(chosen_contexts[word_mask]["context"].apply(aggregate_embs).values)
    mapped_meanings = svd.transform(context_embs)
    clusters = gauss.predict(mapped_meanings)
    chosen_contexts.loc[word_mask, "random_meaning"] = clusters

In [197]:
print(normalized_mutual_info_score(
    chosen_contexts["meaning"],
    chosen_contexts["random_meaning"].astype(float),
))

0.25521153339459823


Зачем даже пытаться... Какая-то у меня злая домашка получилась. Я просто не уважаю ни w2v, ни кластеризацию, ни дизамбигуацию, я занимался в том году через топологию, тут прямо комбо

Короче, мораль такова: усреднять эмбеддинги и молиться, что они что-то дадут, это плохая затея, но почему-то очень популярная

In [212]:
context_embs.shape

(3289, 591052)

In [213]:
meanings["context"]

0       полицейский алекс коллинс и трейся холлоуэй ок...
1       на митинг верующий проходить в киев в время об...
2       президент украина пётр порошенко занять место ...
3       в русский православный церковь рпц назвать кан...
4       абсолютный чемпион мир в первый тяжёлый вес ук...
                              ...                        
6150      4 0 группа человек к рые вместе проводить время
6151    5 0 объединение человек как правило добровольн...
6152    1 0 зд ед способность человек думать лежать в ...
6153    1 1 зд ед высокий степень такой способность вы...
6154    2 0 зд ед мыслительный процесс мозг как место ...
Name: context, Length: 6155, dtype: object

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=1, ngram_range=(1, 2))
context_embs = tfidf.fit_transform(meanings["context"], )
svd = TruncatedSVD(2, random_state=69)

for word in word_dict.word.unique():
    
    word_mask = meanings.word == word
    context_embs = context_embs[word_mask]
    mapped_meanings = svd.fit_transform(context_embs)
    
    n_of_meanings = word_dict[word_dict.word == word].meaning.nunique()
    gauss = GaussianMixture(n_components=n_of_meanings, random_state=69)
    gauss.fit(mapped_meanings)
    
#     # copy, paste...
#     word_mask = chosen_contexts.word == word
#     context_embs = np.vstack(chosen_contexts[word_mask]["context"].apply(aggregate_embs).values)
#     mapped_meanings = svd.transform(context_embs)
#     clusters = gauss.predict(mapped_meanings)
#     chosen_contexts.loc[word_mask, "random_meaning"] = clusters

IndexError: index (6143) out of range